Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [ ]:
#Update your token
STUDENT_TOKEN = 'GROUP 1'

There should be no need to change the code below.

In [ ]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('draw')
      else:
        print('winner' if game.player == player else 'loser')
      return 
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [ ]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class ConnectFour(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):

    for row in self.get_board():
      print(row)
    print("-------------------")
    # print(self.state)

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver"

In [ ]:
def random_solver(game):
  board = game.get_board()
  while True:
      x = 0
      y = random.randint(0, len(board[0]) - 1)
      if board[x][y] == '.':
        print('x: %s, y: %s' % (x, y))
        return y
  return None

In [ ]:
# This function will used to get the index of the row in a given column
def is_valid_move(p, board):
    row_index = -1
    for row in board:
        if row[p] == '.':
            row_index += 1
    return row_index

Column heuristics

In [ ]:
def heuristic_column(column, player, opponent):
    # How many pieces of the same player are in the column and return a score
    count = 0
    column.reverse()
    for i in range(len(column)):
        if column[i] == player:
            count += 1
        # If there is an opponent piece in the column, restart the count
        elif column[i] == opponent:
            count = 0
        # Check if empty space and return the corresponing score 
        elif column[i] == ".":
            if count == 1 and column.count(".") >= 3:
                return 10
            elif count == 2 and column.count(".") >= 2:
                return 20
            else:
                return 0


# This function will check if there is a winner in a column and return the winner
def critical_column(column, player):
    # If the player has a move that creates a 3 in a column in chunks of 4, then it is a winning move
    for i in range(0, len(column) - 3):
        if column[i] == '.' and column[i + 1] == player and column[i + 2] == player and column[i + 3] == player:
            return True

Diagonal heuristic

In [ ]:
# Check positive diagonal
def diagonal_check_pos(version, board, col, player):
    # Loop through the board in a diagonal fashion
    for i in range(0, len(board) - 3):
        for j in range(0, 4):

            # Check for the different versions of critical positions
            for ver in range(len(version)): 
                
                # Check if the position is in the board
                if i + 3 < len(board) and j + 3 < len(board[0]):
                # If the row has 4 consecutive pieces of the same player
                    if board[i][j] == version[0] and board[i + 1][j + 1] == version[1] and board[i + 2][j + 2] == version[2] and board[i + 3][j + 3] == version[3]:
                        
                        # Check if index same as col and if the position is empty
                        for k in range(0,4):
                            if j + k == col and board[i + k][j + k] == ".":

                                # Check if is a valid move and give a count to find if winning move or potential one
                                count = 0
                                for c in range(0, 4):
                                    if is_valid_move(j+c, board) == i+c and board[i+c][j+c] == ".":
                                        count += 1
                                
                                # If the move is a winning move, return the position
                                if count == 1 and version.count(player) == 3:
                                    return True
                                
                                # If the move is a potential winning move, return the position
                                if count == 2 and version.count(player) == 2:
                                    return True
                        
                                # If the move is a good move, return the position
                                if count == 3 and version.count(player) == 1:
                                    return True
                                
                    # Change the version to check for other combinations
                    version = version[1:] + [version[0]]

    # If no winning move is found, return false
    return False


# check 3 in a row in negative diagonal
def diagonal_check_neg(version, board, s, player):
    # Version of critical positions

    # Loop through the board in a diagonal fashion
    for i in range(0, len(board)):
        for j in range(0, 4):

            # Check for the different versions of critical positions
            for p in range(len(version)):
                
                # Check if the position is in the board
                if i -3 >= 0 and j + 3 < len(board[0]):
                    # If the row has 4 consecutive pieces of the same player
                    if board[i][j] == version[0] and board[i - 1][j + 1] == version[1] and board[i - 2][j + 2] == version[2] and board[i - 3][j + 3] == version[3]:
                        
                        # Check if index same as s and if the position is empty
                        for k in range(0,4):
                            if j + k == s and board[i - k][j + k] == ".":

                                # Check if is a valid move and give a count to find if winning move or potential one
                                count = 0
                                for c in range(0, 4):
                                    if is_valid_move(j+c, board) == i-c and board[i-c][j+c] == ".":
                                        count += 1
                            
                                # If the move is a winning move, return the position
                                if count == 1 and version.count(player) == 3:
                                    return True
                                
                                # If the move is a potential winning move, return the position
                                if count == 2 and version.count(player) == 2:
                                    return True
                        
                                # If the move is a good move, return the position
                                if count == 3 and version.count(player) == 1:
                                    return True
                    
                    # Change the version to check for other combinations
                    version = version[1:] + [version[0]]

    # If no winning move is found, return false
    return False

def heuristic_diagonal(board, player, col):
    # Initial variables
    score = 0
    
    # Different versions of potential winning positions
    version1 = [player, ".", player, "."]
    version2 =  [".", player, player, "."]
    version3 = [player, ".", ".", "."]

    # Check positive diagonal
    if diagonal_check_pos(version1, board, col, player):
        score += 20
    
    # Check negative diagonal
    if diagonal_check_neg(version1, board, col, player):
        score += 20

    # Check positive diagonal
    if diagonal_check_pos(version2, board, col, player):
        score += 20 
    
    # Check negative diagonal
    if diagonal_check_neg(version2, board, col, player):
        score += 20

    # Check positive diagonal
    if diagonal_check_pos(version3, board, col, player):
        score += 10

    # Check negative diagonal
    if diagonal_check_neg(version3, board, col, player):
        score += 10

    # Return the score
    return score

def critical_diagonal(board, player, col):

    # Version of critical positions
    version = [player, player, player, "."]

    # Check positive diagonal
    if diagonal_check_pos(version, board, col, player):
        return True

    # Check negative diagonal
    if diagonal_check_neg(version, board, col, player):
        return True

Row heuristic

In [ ]:
# Check the row
def row_score(version, row, col, player):
    # Version of critical positions
    for i in range(0, len(row) - 3):
        # Check for the different versions of critical positions
        
        # Loop through the different versions
        for j in range(0, 4):

            # Check if the position is a type of potential position
            if row[i:i+4] == version:

                # Check if index same as col and if the position is empty
                for k in range(4):
                    if i + k == col and row[i+k] == "." :

                        # Count the number of times a player has consecutive pieces and return critical/potential position
                        if row[i:i+4].count(player) == 3 :
                            return True
                        
                        elif row[i:i+4].count(player) == 2:
                            return True

                        elif row[i:i+4].count(player) == 1:
                            return True
                        
            # Change the version to check for other combinations
            version = version[1:] + [version[0]]

    # If no move is found, return false
    return False

def heuristic_row(row, player, col):
    # Initial variables
    score = 0

    # Different versions of potential winning positions
    version1 = [player, ".", player, "."]
    version2 =  [".", player, player, "."]
    version3 = [player, ".", ".", "."]
    
    # Check the row
    if row_score(version1, row, col, player):
        score += 20

    # Check the row
    if row_score(version2, row, col, player):
        score += 20

    # Check the row
    if row_score(version3, row, col, player):
        score += 10
        
    # Return the score
    return score


def critical_row(row, player, col):

    # Version of critical positions
    version = [player, player, player, "."]

    # Check the row
    if row_score(version, row, col, player):
        return True

In [ ]:
# Algorithm to play Connect 4 using heuristics

# Heuristic for the Connect Four game
# Maximize the number of consecutive pieces in a row for the player and minimize for the opponent
# Maximize the number of consecutive pieces in a column for the player and minimize for the opponent
# Maximize the number of consecutive pieces in a diagonal for the player and minimize for the opponent


# j is the index of the last piece played
def heuristic(game, j):
    # Initial variables
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    score = 0    

    # Get the column of the last piece played
    column = [board[i][j] for i in range(len(board))]

    # Get the row of the last piece played
    row = board[is_valid_move(j, board)]

    # Early stoping if the column is full
    if column.count(".") == 0:
        return 0

    # Critical winning moves
    if critical_column(column, player):
        return 10000

    if critical_diagonal(board, player, j):
        return 10000

    if critical_row(row, player, j):
        return 10000

    # Critical losing moves
    if critical_column(column, opponent):
        return 5000

    if critical_diagonal(board, opponent, j):
        return 5000

    if critical_row(row, opponent, j):
        return 5000


    # Check columns
    score += heuristic_column(column, player, opponent)

    # Check Diagonals
    score += heuristic_diagonal(board, player, j)

    # Check Rows
    score += heuristic_row(row, player, j)

    return score

In [ ]:
# Algorithm to find the best move by using the heuristic function 
# This is an minimax algorithm that uses the heuristic function to find the best move
# Go to depth 5 and then use the heuristic function to find the best move

# Do minimax with the heuristic function
def minimax(game, depth, is_maximizing, alpha, beta, col):
    # Initial variables
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    best_score = 0

    # Check if the game is over
    if game is None:
        if game.winner() == player:
            return 10000
        else:
            return 0

    # Check if the depth is 0
    if depth == 0:
        return heuristic(game, col)

    # Check if it is the maximizing player
    if is_maximizing:
        best_score = -100000

        # Loop through the columns to find the best move for the player
        for i in range(len(board[0])):
            if board[0][i] == ".":
                board[0][i] = player

                # Give a score to the move
                score = minimax(game, depth - 1, False, alpha, beta, col)
                board[0][i] = "."

                # Check if the score is better than the best score and update alpha
                best_score = max(score, best_score)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
        
        # Return the best score
        return best_score

    # Check if it is the minimizing player
    else:
        best_score = 100000

        # Loop through the columns to find the best move for the opponent
        for i in range(len(board[0])):
            if board[0][i] == ".":
                board[0][i] = opponent

                # Give a score to the move
                score = minimax(game, depth - 1, True, alpha, beta, col)
                board[0][i] = "."

                # Check if the score is better than the best score and update beta
                best_score = min(score, best_score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break
        return best_score

def best_move(game):
    # Initial variables
    board = game.get_board()
    player = game.player
    opponent = game.other_player()
    best_score = -100000
    best_move = 0
    depth = 5

    # We will store all the scores to find the best move
    score_table = []
    
    # Go depth 5
    for i in range(len(board[0])):
        if board[0][i] == ".":
            board[0][i] = player

            # Give a score to the move using the mibnimax algorithm with a heuristic function
            score = minimax(game, depth - 1, False, -100000, 100000, i)
            board[0][i] = "."

            # Store all scores in a table
            score_table.append((score, i))

            # Find the best score and the best move
            if score > best_score:
                best_score = score
                best_move = i

    # When score is the same as the best score, then choose a random move
    # Extract the moves that have the same score as the best score
    best_moves = [move for score, move in score_table if score == best_score]

    print(score_table)
    # Choose a random move from the best moves
    best_move = random.choice(best_moves)

    # Return the best move
    return best_move

Single Game against Random Solver

In [ ]:
game_loop(best_move, ConnectFour, 'connect4big', multi_player=False)

Tests

In [ ]:
# Loop of 100 games and count the number of wins
def play_game():
    wins = 0
    for i in range(100):
        game_loop(best_move, ConnectFour, 'connect4big', multi_player=False)
            
# Check wins by searchin 'winner' in the output

play_game()

# 1 try on V2, 85% of wins
# 2 try on V2, 96% of wins
# 3 try on V3, 90% of wins (takes into account diagonal)
# 4 try on V4, 99% of wins
# 5 try on V5, 99% of wins (Random moves, eventually makes check mate)